## Session Setup

In [1]:
# Check to make sure there are multiple gpus available
import torch, os
!export CUDA_LAUNCH_BLOCKING=1

import gc, torch, os
torch.cuda.empty_cache()
gc.collect()

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"
os.chdir("/mnt/home/amir/framingdecomp/framingDecomp")

print("Devices visible:", os.environ.get("CUDA_VISIBLE_DEVICES"))
print("torch.cuda.device_count():", torch.cuda.device_count())


Devices visible: 4,5
torch.cuda.device_count(): 0


/mnt/home/amir/jupyter-env/lib/python3.11/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
# ==== Cell: [Session setup] ====

import os, sys, logging, random, yaml, time, uuid, json
from pathlib import Path

import torch
import numpy as np

# ——— switches you may tune ——————————————————————————
USE_MULTIGPU     = True          # False → single-GPU
# VISIBLE_DEVICES  = "0,1,2,3"
# CFG_PATH         = "configs/decomposer3.yaml"
CFG_PATH         = "configs/decomposer_main.yaml"
# ————————————————————————————————————————————————


# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Device: {device}, GPUs: {torch.cuda.device_count()}")

# ---------- logging ----------
Path("logs").mkdir(exist_ok=True)
ts = time.strftime("%Y%m%d_%H%M%S")
log_path = Path(f"logs/decomposer_{ts}.log")

# root logger -> both console and file
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s — %(levelname)s — %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(log_path, mode="w")
    ]
)
logger = logging.getLogger("train_decomposer")
logger.info("Log file created at %s", log_path)


# --- config, seeds, logging -----------------------------------
with open(CFG_PATH, "r") as f:
    config = yaml.safe_load(f)

seed = config["experiment"]["seed"]
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

logger.info("Config loaded and seeds set.")

2025-07-29 23:49:40,990 — INFO — Log file created at logs/decomposer_20250729_234940.log
2025-07-29 23:49:40,994 — INFO — Config loaded and seeds set.


## Data Loading and Preprocessing

In [3]:
# ==== Cell: [Data loading & preprocessing] ====

import json
from typing import List, Dict

def load_jsonl(path: str) -> List[Dict]:
    with open(path, "r") as f:
        return [json.loads(l) for l in f if l.strip() and not l.strip().startswith("#")]

data_cfg = config["data"]

raw_F_harm  = load_jsonl(data_cfg["input_path_varyFraming"])
raw_G_harm  = load_jsonl(data_cfg["input_path_varyGoal"])
raw_F_ben   = load_jsonl(data_cfg["input_path_varyFraming_benign"])
raw_G_ben   = load_jsonl(data_cfg["input_path_varyGoal_benign"])

for e in raw_F_harm + raw_F_ben: e["split"] = "varyF"
for e in raw_G_harm + raw_G_ben: e["split"] = "varyG"

def _preprocess(entries: List[Dict], max_f_idx: int):
    processed = []
    for ent in entries:
        req = ["prompt","goal","goal_index","framing_index","split"]
        if not all(k in ent for k in req): 
            continue
        g, f = ent["goal_index"], ent["framing_index"]
        if ent["split"] == "varyF":                 # re-index framings
            f = g if f == 0 else max_f_idx + 1
            max_f_idx = max(max_f_idx, f)
        processed.append({
            "text":  ent["prompt"],
            "goal":  ent["goal"],
            "goal_index": g,
            "framing_index": f,
            "label": ent.get("jailbroken", False),
            "split": ent["split"],
        })
    return processed, max_f_idx

max_idx = max(e["framing_index"]
              for e in raw_F_harm + raw_G_harm + raw_F_ben + raw_G_ben)

P_F_harm, max_idx = _preprocess(raw_F_harm, max_idx)
P_G_harm, max_idx = _preprocess(raw_G_harm, max_idx)
P_F_ben , max_idx = _preprocess(raw_F_ben , max_idx)
P_G_ben , max_idx = _preprocess(raw_G_ben , max_idx)

all_samples = P_F_harm + P_G_harm + P_F_ben + P_G_ben
n_total = len(all_samples)
n_sample = int(n_total * config.get('sample_prop', 1.))
n_sample = max(n_sample, 500)
if n_sample < n_total:
    logger.info("Sampling %d out of %d total samples", n_sample, n_total)
    all_samples = random.sample(all_samples, n_sample)
logger.info("Total processed samples: %d", len(all_samples))

2025-07-21 20:48:58,730 — INFO — Total processed samples: 5286


## Dataloader

In [4]:
# ==== Cell: [Dataset & dataloader] ====

from collections import defaultdict
from typing import Tuple
from torch.utils.data import Dataset, DataLoader

class DualPairDataset(Dataset):
    """
    Returns (sample_a, sample_b, pair_type)
      pair_type = 0 → same-goal / diff-frame  (from varyF)
      pair_type = 1 → same-frame / diff-goal  (from varyG)
    """
    def __init__(self, samples, stratified_capping=True):
        self.samples = samples
        self.goal_pairs, self.frame_pairs = [], []

        by_goal_F  = defaultdict(list)
        by_frame_G = defaultdict(list)

        for idx, s in enumerate(samples):
            if s["split"] == "varyF":  by_goal_F [s["goal_index"]   ].append(idx)
            else:                      by_frame_G[s["framing_index"]].append(idx)

        for lst in by_goal_F.values():
            self.goal_pairs  += [(a,b,0) for a in lst for b in lst if a<b]
        for lst in by_frame_G.values():
            self.frame_pairs += [(a,b,1) for a in lst for b in lst if a<b]

        # --- stratified capping ---------------------------------
        # this improved the performance a bit
        if stratified_capping:
            cap = int(np.median([len(v) for v in by_goal_F.values()]))
            for g, lst in by_goal_F.items():
                if len(lst) > cap:               # down-sample heavy goals
                    by_goal_F[g] = random.sample(lst, cap)
        # --------------------------------------------------------------

        self.all_pairs = self.goal_pairs + self.frame_pairs

    def __len__(self): return len(self.all_pairs)
    def __getitem__(self, k): return self.all_pairs[k]

def collate_dual(batch) -> Tuple[list,str,str,torch.Tensor]:
    """
    batch → (texts, goal_ids, frame_ids, pair_types)
    """
    texts, gid, fid, ptype = [], [], [], []
    for a,b,t in batch:
        sa, sb = all_samples[a], all_samples[b]
        texts.extend([sa["text"], sb["text"]])
        gid.extend([sa["goal_cid"], sb["goal_cid"]])
        fid.extend([sa["framing_index"], sb["framing_index"]])
        ptype.append(t)
    return (texts,
            torch.tensor(gid),
            torch.tensor(fid),
            torch.tensor(ptype))

# contiguous goal ids
unique_goals = sorted({s["goal_index"] for s in all_samples})
goal2cid     = {g:i for i,g in enumerate(unique_goals)}
for s in all_samples: s["goal_cid"] = goal2cid[s["goal_index"]]

train_ds = DualPairDataset(all_samples)
logger.info("Goal pairs: %d   Frame pairs: %d   Total pairs: %d",
            len(train_ds.goal_pairs),
            len(train_ds.frame_pairs),
            len(train_ds))

2025-07-21 20:48:58,761 — INFO — Goal pairs: 86824   Frame pairs: 89419   Total pairs: 176243


## Training and Launcher

In [5]:
# ==== Cell: [Training worker & launch] ====

from accelerate import notebook_launcher
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)
from utils.misc import seed_worker, set_seed


MODEL_JOB_PREFIXES = {
    "meta-llama/Llama-3-8B-Instruct": "ll3",
    "meta-llama/Llama-2-7b-chat-hf": "ll2", 
    "lmsys/vicuna-13b-v1.5": "vic13",
    "lmsys/vicuna-7b-v1.5": "vic7",
    "mistralai/Mistral-7B-v0.1": "mis",
    "deepseek-ai/deepseek-llm-7b-chat": "dsk",
    "gpt2-medium": "gpt2",
    "google/gemma-2-9b": "gem9",
    "google/gemma-2-2b": "gem2"
}

ts = None
run_id = None
global run_id, ts, MODEL_JOB_PREFIXES

def train_worker():
    import gc, yaml, torch, torch.distributed as dist
    from torch.optim import AdamW
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from torch.nn.parallel import DistributedDataParallel as DDP
    from train_test.decomposer_training import train_decomposer
    from utils.model_utils import load_model_multiGPU
    from models.encoder import HFEncoder_notPooled
    from models.decomposer import NonlinearDecomposer, NonlinearDecomposer_tiny

    gc.collect(); torch.cuda.empty_cache()

    local_rank = int(os.environ["LOCAL_RANK"])
    world_size = int(os.environ.get("WORLD_SIZE", "1"))
    distributed = world_size > 1

    if distributed:
        dist.init_process_group(
            backend="nccl", init_method="env://",
            rank=local_rank, world_size=world_size,
        )

    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)

    # ——— load LLM once per rank ———
    model, tokenizer = load_model_multiGPU(
        model_name=config["model"]["name"],
        local_rank=local_rank,
        load_in_8bit=False,
        load_in_4bit=False,
    )
    logger.info("Loaded model %s ", config["model"]["name"])
    
    config["model"]["layers"] = 'all'
    if config["model"]["layers"] == 'all':
        num_layers = model.config.num_hidden_layers
        layers = list(range(num_layers))
    else:
        layers = config["model"]["layers"]
        if isinstance(layers, int): layers = [layers]
    
    # --- Set arguments # TODO: Delete and instead fix the config
    config['experiment']['use_sae'] = config['experiment'].get('use_sae', False)  # use Sparse Autoencoder
    config['training']['num_epochs'] = 3
    init_lambda_orth = config['lambda_orth']
    config['lambda_repulse'] = config.get('lambda_repulse', 6.0) 
    config['lambda_adv'] = config.get('lambda_adv', 2.0)  
    config['lambda_sparse'] = config.get('lambda_sparse', None)  
    config['lambda_recon'] = config.get('lambda_recon', 1.0)  
    config['lambda_Worth'] = config.get('lambda_Worth', 0.05)
    config['grad_accum_steps'] = config.get('grad_accum_steps', 8)
    config['model']['layers'] = [-1]#layers
    config['model']['layer_combine'] = config['model'].get('layer_combine', 'mean')
    config['model']['last_token'] = config['model'].get('last_token', False)
    config['training']['num_epochs']   = 3
    config['training']['batch_size']   = 8
    # ---
    
    for layer in layers:
        torch.manual_seed(config['experiment']['seed'])
        torch.cuda.manual_seed_all(config['experiment']['seed'])
        np.random.seed(config['experiment']['seed'])
        random.seed(config['experiment']['seed'])
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
        logger.info(f"Training decomposer for layer {layer}")
        encoder = HFEncoder_notPooled(
            model=model,
            tokenizer=tokenizer,
            device=device,
            layers=[layer],
            layer_combine=config["model"].get("layer_combine", "mean"),
            last_token=config["model"].get("last_token", True),
        )
        encoder.eval()

        # ——— decomposer ———
        enc_dim = model.config.hidden_size
        dec = NonlinearDecomposer(
            enc_dim=enc_dim,
            d_g=config["d_g"],
            d_f=config["d_f"],
            hidden_dim=config.get("hidden_dim", 1024),
            dropout=config.get("dropout", 0.1),
        ).to(device)
        d_f = dec.Wf(torch.randn(4096).to(device)).shape[0]
        dec = DDP(dec, device_ids=[local_rank])

        # ——— dataloader ———
        from torch.utils.data import DistributedSampler
        sampler = DistributedSampler(train_ds, rank=local_rank,
                                        num_replicas=world_size, shuffle=True, seed=config['experiment']['seed'])
        gen = torch.Generator()
        gen.manual_seed(config['experiment']['seed']) 
        train_loader = DataLoader(
            train_ds,
            batch_size=config["training"]["batch_size"],
            sampler=sampler,
            collate_fn=collate_dual,
            num_workers=8,
            pin_memory=True,
            shuffle=False,
            worker_init_fn=lambda wid: set_seed(config['experiment']['seed'] + wid),#seed_worker,
            generator=gen,
        )

        # ——— optim & sched ———
        opt = AdamW(dec.parameters(), lr=config["lr"])
        sched = CosineAnnealingLR(opt,
                                    T_max=len(train_ds)*config["training"]["num_epochs"])

        # adversarial classifier
        n_goals = len(unique_goals)
        adv_clf = torch.nn.Linear(d_f, n_goals).to(device)
        adv_clf = DDP(adv_clf, device_ids=[local_rank])
        adv_opt = AdamW(adv_clf.parameters(), lr=1e-4)

        # ——— train ———
        stats = train_decomposer(
            encoder     = encoder,
            decomposer  = dec,
            dataloader  = train_loader,
            optimizer   = opt,
            adv_clf     = adv_clf,
            adv_opt     = adv_opt,
            lambda_adv  = config.get('lambda_adv', 2.0),
            scheduler   = sched,
            device      = device,
            epochs      = config['training']['num_epochs'],
            lambda_g    = config['lambda_g'],
            lambda_f    = config['lambda_f'],
            lambda_repulse = config.get('lambda_repulse', 6.0),  # optional
            lambda_orth = config['lambda_orth']*10,
            lambda_recon = config.get('lambda_recon', 1.0),  # optional
            lambda_Worth = config.get('lambda_Worth', 0.25),
            grad_clip   = config['grad_clip'],
            grad_accum_steps = config["training"].get('grad_accum_steps', 4),
            log_every   = 50,
            info        = logger.info,
            layer_str= f"{layer}",
        )

        # ——— checkpoint (rank-0 only) ———
        if local_rank == 0:
            # ts = time.strftime("%Y%m%d_%H%M%S")
            # run_id = str(uuid.uuid4())
            global run_id, ts, MODEL_JOB_PREFIXES
            model_short = MODEL_JOB_PREFIXES.get(config["model"]["name"], "unknownModel")
            ckpt_dir = Path(f"checkpoints/decomposer_simple/{model_short}_decomposer_layer{layer}_{ts}_{run_id}")
            ckpt_dir.mkdir(parents=True, exist_ok=True)
            
            if layer == 0:
                config_filename = f"./output/config_{model_short}_{ts}_{run_id}.yaml"
                with open(config_filename, "w") as f:
                    yaml.safe_dump(config, f)

            torch.save({k: v.cpu() for k, v in dec.module.state_dict().items()},
                        ckpt_dir / "weights.pt")
            with open(ckpt_dir / "train_stats.json", "w") as f:
                json.dump(stats, f)
            logger.info("Checkpoint for layer %d saved to %s", layer, ckpt_dir)

    if distributed:
        dist.destroy_process_group()



In [ ]:
# ——— launch ———
# from random import randint
# os.environ["MASTER_PORT"] = str(15000 + randint(0, 10000))

# num_proc = torch.cuda.device_count() if USE_MULTIGPU else 1

global run_id, ts
ts = time.strftime("%Y%m%d_%H%M%S")
run_id = str(uuid.uuid4())
num_proc=2
notebook_launcher(train_worker, num_processes=num_proc)

Launching training on 2 GPUs.


[W721 20:48:59.545498356 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W721 20:48:59.545846066 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W721 20:49:02.089468330 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W721 20:49:02.089938455 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W721 20:49:02.108150847 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W721 20:49:02.108386983 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
/mnt/home/amir/jupyter-env/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/mnt/home/amir/jupyter-env/lib/python3.11/site-packages/transformers/models/auto/token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

epoch 0, layer 2:  85%|█████████████████████████████████████████████████████████████           | 9349/11016 [23:45<04:20,  6.40it/s, loss=5.68]

2025-07-22 00:00:48,650 — INFO — layer 2, [0:9349], loss=4.8619
2025-07-22 00:00:48,650 — INFO — layer 2, [0:9349], loss=4.7906


epoch 0, layer 2:  85%|██████████████████████████████████████████████████████████████▎          | 9399/11016 [23:53<04:14,  6.35it/s, loss=4.8]

2025-07-22 00:00:56,277 — INFO — layer 2, [0:9399], loss=5.0543
2025-07-22 00:00:56,278 — INFO — layer 2, [0:9399], loss=5.1309


epoch 0, layer 2:  86%|█████████████████████████████████████████████████████████████▌          | 9427/11016 [23:57<04:07,  6.43it/s, loss=7.31]

In [8]:
logger

<Logger train_decomposer (INFO)>